In [2]:
import numpy as np
from itertools import product

In [19]:
class Schelling:

    def __init__(self, size:int, N:int, jr:float, jb:float, offset:int):
        """
            size - number of locations in the grid,
            N - number of agents,
            jr - similar neighbor index for red agents,
            jb - similar neighbor index for blue agents,
            offset - neighborhood radius, number of neighbors will be (2*offset + 1)**2 - 1
        """

        self.grid = None
        self.gameGrids = None
        self.size = size
        self.L = int(np.sqrt(size))
        self.N = N
        self.jr = jr
        self.jb = jb
        self.offset = offset
        self.mt = (2*offset + 1)**2 - 1

    def initGrid(self):

        grid = np.zeros(self.size, dtype=int)
        indices = list(range(self.size))
        populated = np.random.choice(indices, self.N, replace=False)
        red = populated[:int(self.N/2)]
        blue = populated[int(self.N/2):]
        grid[red] = 1
        grid[blue] = 2
        self.grid = grid.reshape((self.L, self.L))

    def getNeighbors(self, row, col):
        
        L = self.grid.shape[0]

        row_offsets = list(range(-self.offset, self.offset+1))
        col_offsets = row_offsets
        neighbors = {(row+r, col+c) for r, c in product(row_offsets, col_offsets) if 0 <= row+r < L and 0 <= col+c < L}
        neighbors.discard((row, col))

        return list(neighbors)
    
    def getSimilarNeighbors(self, row, col):
        
        color = self.grid[row, col]
        neighbors = self.getNeighbors(row, col)
        similar = [n for n in neighbors if self.grid[n[0], n[1]] == color]
        
        return len(similar)
    
    def simulate(self):
        
        occupied = list(zip(*np.where(self.grid != 0)))
        occ_indices = list(range(len(occupied)))
        free = list(zip(*np.where(self.grid == 0)))
        free_indices = list(range(len(free)))
        jts = {1: self.jr, 2: self.jb}

        self.gameGrids = np.copy(self.grid)
        ts = 0

        while True:
            grid = self.grid

            for _ in range(self.N):

                agent_ind = np.random.choice(occ_indices)
                agent = occupied[agent_ind]
                color = grid[agent[0], agent[1]]
                jt = jts[color]

                if self.getSimilarNeighbors(agent[0], agent[1]) / self.mt < jt:

                    new_ind = np.random.choice(free_indices)
                    new = free[new_ind]
                    grid[agent[0], agent[1]] = 0
                    grid[new[0], new[1]] = color
                    occupied[agent_ind] = new
                    free[new_ind] = agent

            self.gameGrids = np.vstack((self.gameGrids, grid))
            self.grid = grid
            ts += 1

            if np.array_equal(self.gameGrids[ts*self.L:(ts+1)*self.L, :], self.gameGrids[(ts-1)*self.L:(ts)*self.L, :]):
                break   

In [37]:
ss = Schelling(10**2, 30, 1/3, 1/3, 1)

In [38]:
ss.initGrid()

In [39]:
ss.grid

array([[0, 1, 0, 0, 0, 0, 0, 1, 2, 0],
       [0, 0, 1, 0, 0, 2, 0, 0, 0, 0],
       [0, 1, 2, 0, 2, 0, 2, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 2, 1, 0, 2],
       [0, 0, 1, 2, 0, 0, 0, 0, 2, 0],
       [0, 0, 0, 2, 0, 1, 0, 0, 0, 0],
       [2, 0, 0, 2, 1, 0, 0, 0, 0, 0],
       [1, 2, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 2, 0, 0, 1, 0, 2, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [41]:
ss.simulate()

In [44]:
ss.gameGrids[-10:-1, :]

array([[0, 0, 1, 1, 0, 0, 0, 1, 1, 0],
       [0, 1, 1, 1, 2, 0, 0, 1, 1, 1],
       [0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 2, 2, 0],
       [0, 0, 2, 0, 0, 2, 2, 2, 2, 0],
       [0, 0, 0, 0, 0, 0, 2, 2, 2, 0]])

In [12]:
a[b] = 1
a

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [14]:
a.reshape((10, 10))

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])